In [1]:
import os
import json
import pickle
import numpy as np
import cv2
from keras.models import load_model
from tensorflow.keras.utils import img_to_array, load_img
from PIL import ImageFile


In [2]:
BASE_MODEL_PATH = "/.model"
TEST_DIR = "csv_files/est.csv"
PREDICT_DIR = "./pred_dir"
PICKLE_DIR = "./pickle_files"
JSON_DIR = "./json_files"

In [ ]:
if not os.path.exists(JSON_DIR):
    os.makedirs(JSON_DIR)

In [ ]:
BEST_MODEL = ".\model\self-trained\CNN_non_batch_best_model.keras"
model = load_model(BEST_MODEL)
model.summary()

In [ ]:
data_test = pd.read_csv(os.path.join(TEST_DIR))
#testing on the only 10000 images as loading the all test images requires ram>8gb
data_test = data_test[:10000] 
data_test.info()